In [8]:
import pandas as pd
import numpy as np

# === Step 0: Update this path to your CSV if needed ===
csv_path = "Dataset\heart_disease_uci.csv"  
output_file = "heart_case_study_cleaned.xlsx"

# === Step 1: Load dataset ===
df = pd.read_csv(csv_path)
print("Original shape:", df.shape)
print("Columns:", df.columns.tolist())


Original shape: (920, 16)
Columns: ['id', 'age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']


In [9]:
# === Step 2: Clean column names ===
df.columns = [c.strip().lower() for c in df.columns]

# Rename columns if they exist
rename_dict = {
    "age": "age",
    "sex": "sex",
    "cp": "chest_pain",
    "trestbps": "resting_bp",
    "chol": "cholesterol",
    "fbs": "fasting_bs",
    "restecg": "rest_ecg",
    "thalach": "max_hr",
    "exang": "exercise_angina",
    "oldpeak": "st_depression",
    "slope": "st_slope",
    "ca": "vessels",
    "thal": "thal",
    "num": "target",
    "target": "target",
    "id": "id"
}

rename_subset = {k: v for k, v in rename_dict.items() if k in df.columns}
df = df.rename(columns=rename_subset)


In [10]:
# === Step 3: Replace common missing tokens and convert numeric cols ===
df.replace(["?", "NA", "N/A", ""], np.nan, inplace=True)

for col in ["age","resting_bp","cholesterol","max_hr","st_depression","vessels"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")



In [11]:
# === Step 4: Simple cleaning strategy ===
essential = [c for c in ["age","sex","resting_bp","cholesterol","max_hr","target"] if c in df.columns]
print("Essential columns detected:", essential)

df_before = df.shape[0]
df = df.dropna(subset=essential)
print(f"Dropped {df_before - df.shape[0]} rows with missing essential values. New shape: {df.shape}")


Essential columns detected: ['age', 'sex', 'resting_bp', 'cholesterol', 'target']
Dropped 86 rows with missing essential values. New shape: (834, 16)


In [12]:
# === Step 5: Standardize some categorical encodings ===
if "sex" in df.columns:
    if pd.api.types.is_numeric_dtype(df["sex"]):
        vals = set(df["sex"].dropna().astype(int).unique())
        if vals.issubset({0,1}):
            df["sex"] = df["sex"].map({1:"Male", 0:"Female"})
        elif vals.issubset({1,2}):
            df["sex"] = df["sex"].map({1:"Male", 2:"Female"})
        else:
            df["sex"] = df["sex"].astype(str)
    else:
        df["sex"] = df["sex"].astype(str).str.capitalize()

if "fasting_bs" in df.columns:
    df["fasting_bs"] = df["fasting_bs"].map({1:True, 0:False}).fillna(df["fasting_bs"])

if "exercise_angina" in df.columns:
    df["exercise_angina"] = df["exercise_angina"].map({1:True, 0:False}).fillna(df["exercise_angina"])


C:\Users\Divyaa Sriram\AppData\Local\Temp\ipykernel_31020\1241260708.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["exercise_angina"] = df["exercise_angina"].map({1:True, 0:False}).fillna(df["exercise_angina"])


In [13]:
# === Step 6: Create age_group for ANOVA ===
if "age" in df.columns:
    bins = [0,29,45,60,200]
    labels = ["<30","30-45","46-60",">60"]
    df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, include_lowest=True)


In [14]:
# === Step 7: Build README variable dictionary safely ===
desc_map = {
    "id": "Unique patient identifier (if present)",
    "age": "Age in years",
    "sex": "Sex (Male/Female)",
    "chest_pain": "Chest pain type (categorical)",
    "resting_bp": "Resting blood pressure (mm Hg)",
    "cholesterol": "Serum cholesterol (mg/dl)",
    "fasting_bs": "Fasting blood sugar > 120 mg/dl (True/False)",
    "rest_ecg": "Resting electrocardiographic results (categorical)",
    "max_hr": "Maximum heart rate achieved",
    "exercise_angina": "Exercise-induced angina (True/False)",
    "st_depression": "ST depression induced by exercise",
    "st_slope": "Slope of the peak exercise ST segment",
    "vessels": "Number of major vessels colored by fluoroscopy (0-3)",
    "thal": "Thalassemia status (categorical)",
    "target": "Heart disease diagnosis (0 = no disease, >0 = disease)",
    "age_group": "Derived age group for ANOVA"
}

var_rows = []
for col in df.columns:
    desc = desc_map.get(col, "No description available (check original dataset documentation)")
    var_rows.append((col, desc))

var_df = pd.DataFrame(var_rows, columns=["Variable", "Description"])


In [15]:
# === Step 8: Dataset info frame ===
dataset_info = pd.DataFrame({
    "Attribute": ["Dataset Name", "Source", "Original file", "Rows (after cleaning)", "Columns kept", "Cleaning notes"],
    "Value": [
        "Cleveland Heart Disease (subset)",
        "UCI Repository / Kaggle",
        csv_path,
        str(df.shape[0]),
        ", ".join(df.columns.tolist()),
        "Standardized names, dropped rows with missing essential values, converted numeric types where possible."
    ]
})

# === Step 9: Numeric summary & categorical counts (for Summary sheet) ===
summary_numeric = df.select_dtypes(include=[np.number]).describe().T.reset_index().rename(columns={"index":"variable"})
cat_counts = {}
for c in df.select_dtypes(exclude=[np.number, "datetime"]).columns:
    if df[c].nunique() <= 50:  
        cat_counts[c] = df[c].value_counts(dropna=False).reset_index().rename(columns={"index": c, 0:"count"})


In [16]:
# === Step 10: Save to Excel ===
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    dataset_info.to_excel(writer, sheet_name="README", index=False, startrow=0)
    var_df.to_excel(writer, sheet_name="README", index=False, startrow=8)
    df.to_excel(writer, sheet_name="CleanedData", index=False)
    summary_numeric.to_excel(writer, sheet_name="Summary", index=False, startrow=0)
    row = len(summary_numeric) + 3
    for cname, cdf in cat_counts.items():
        cdf.to_excel(writer, sheet_name="Summary", index=False, startrow=row)
        row += len(cdf) + 3

print(f"Saved cleaned Excel to: {output_file}")
print("\nSample of cleaned data:")
print(df.head().to_string(index=False))
print("\nVariable dictionary preview:")
print(var_df.to_string(index=False))

Saved cleaned Excel to: heart_case_study_cleaned.xlsx

Sample of cleaned data:
 id  age    sex   dataset      chest_pain  resting_bp  cholesterol fasting_bs       rest_ecg  thalch  exercise_angina  st_depression    st_slope  vessels              thal  target age_group
  1   63   Male Cleveland  typical angina       145.0        233.0       True lv hypertrophy   150.0            False            2.3 downsloping      0.0      fixed defect       0       >60
  2   67   Male Cleveland    asymptomatic       160.0        286.0      False lv hypertrophy   108.0             True            1.5        flat      3.0            normal       2       >60
  3   67   Male Cleveland    asymptomatic       120.0        229.0      False lv hypertrophy   129.0             True            2.6        flat      2.0 reversable defect       1       >60
  4   37   Male Cleveland     non-anginal       130.0        250.0      False         normal   187.0            False            3.5 downsloping      0.0        